# Trend following con Histograma MACD

Se utiliza el indicador técnico seguidor de tendencia denominado "MACD" (Moving Average Convergence Divergence) para determinar tendencia y cambios de estados (señales) en la situación de activos de renta variable o índices. Tres subsistemas según compresión temporal (diaria, semanal, mensual): MACD Mensual, MACD Semanal, MACD Diario.

## INDICE DE FUNCIONES:
- analyzeMACD(activos, timeframe):
    Esta función analiza los activos solicitados con el indicador técnico MACD y calcula la tendencia, dominancia y fortaleza de la 
    dominancia. Definiciones:
        - "Tendencia": si la línea MACD es mayor a 0, la tendencia es alcista, y si la misma es menor a 0, es bajista.
        - "Dominancia": si la línea MACD es mayor a la línea señal, la dominancia es mayor a 0 (histograma positivo), y viceversa.
        - "Fortaleza": - Si la dominancia es alcista: si la diferencia entre las líneas MACD y señal se hace más positiva,
                       la fortaleza es Creciente, y sino, Decreciente.
                       - Si la dominancia es bajista: si la diferencia entre las líneas MACD y señal se hace más negativa,
                       la fortaleza es Creciente, y sino, Decreciente.
    Se pueden consultar sets de activos preconfigurados: "cedears", "adrs", "sectores", "lideres", "general".
    
- analyzeFullMACD(activos):
    Esta función se apoya en la función analyze MACD para generar un análisis multitemporal de los activos especificados.
    Los timeframes analizados son: mensual, semanal y diario. Se pueden consultar sets de activos preconfigurados: "cedears", "adrs", "sectores", "lideres", "general". Devuelve una tabla con estilos rojos/verdes representativos de los estados.
 
- getMACDSignals(activos, timeframe):
    Esta función descarga data financiera de los activos seleccionados, calcula el indicador técnico MACD e identifica las últimas señales vigentes de compra (long) o bien de
    venta en corto (short), devolviendo la fecha en que se produjo la señal, el precio, el rendimiento acumulado al día de la consulta y el rendimiento anualizado. Estos datos
    se devuelven en formato diccionario o dataframe (tabla) para todos los activos consultados. Las señales son en función de dos tipos de criterios:
        - Cruce de la línea MACD de la posición 0, que es equivalente a un crossover de las medias móviles exponenciales de 12 y 26 ruedas;
        - Cruce de la línea MACD a la línea Señal (crossover), siendo la línea Señal una EMA de 9 ruedas de la línea MACD.

In [1]:
# FUNCIONES NECESARIAS PARA OBTENER DATA FINANCIERA:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

def getDataYf(ticker, tipo, interval, data_from = None, data_to = None, period = None):
    """
    Es una función para descargar market data de Yahoo Finance con la librería yfinance.
    
    ## Inputs:
        >ticker: el nombre del ticker.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    if tipo == "no end":
        data = yf.download(ticker, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(ticker, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(ticker, interval = interval, period = period, progress = False, auto_adjust = True)
    return data



def getDataYfMulti(activos, tipo, interval, data_from = None, data_to = None, period = None, swap = True):
    """
    Función para hacer batch requests (varios tickers a la vez), que será la fx que más voy a utilizar para market data.
    
    ## Inputs:
        >tickers: es una lista con los tickers de los cuales se va a obtener market data.
        >tipo: si es "no end" no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si es "end" es
        necesario indicar hasta cuánto (data_to). En ambos casos hay que indicar desde qué fecha (data_from). Si es "period" 
        no se indica ni desde cuándo ni hasta cuándo, sólo el argumento "period" con la cantidad de tiempo a obtener.
        >now : si es True, no se indica hasta cuándo (data_to), se obtiene hasta el último día disponible. Si se indica False, es
        necesario indicar hasta cuánto (data_to).
        >interval: el timeframe (ej. 1mo, 1h, 1d, 1wk, etc)
        >data_from: data desde qué fecha.
        >data_to: data hasta qué fecha (no inclusive el día). Sólo es aplicable si now == True.
        >period : en caso de tipo = "period", se pasa este argumento que refiere a la cantidad de tiempo a obtener. Ej. 1y, 2y, 3y, etc.
        >swap : si es True, divide el df en tickers y cada uno tiene su OHLC. Si es False, tenemos cada columna OHLC y dentro todos los tickers.
        
    ## Outputs:
        >series OHLC ajustadas del ticker.
    """
    import yfinance as yf
    import pandas as pd
    
    lideres_arg = ["ALUA.BA", "BBAR.BA", "BMA.BA", "BYMA.BA", "CEPU.BA", "COME.BA", "CRES.BA", "CVH.BA", "EDN.BA", 
                   "GGAL.BA", "LOMA.BA", "MIRG.BA", "PAMP.BA", "SUPV.BA", "TECO2.BA", "TGNO4.BA", "TGSU2.BA", "TRAN.BA", 
                   "TXAR.BA", "VALO.BA", "YPFD.BA"]

    general_arg = ["AGRO.BA", "AUSO.BA", "BHIP.BA", "BOLT.BA", "BPAT.BA", "CADO.BA", "CAPX.BA", "CARC.BA", "CECO2.BA", 
                   "CELU.BA", "CGPA2.BA", "CTIO.BA", "DGCU2.BA", "FERR.BA", "FIPL.BA", "GAMI.BA", "GCDI.BA", "GCLA.BA", 
                   "HARG.BA", "HAVA.BA", "INVJ.BA", "IRSA.BA", "LEDE.BA", "LONG.BA", "METR.BA", "MOLA.BA", "MOLI.BA", 
                   "MORI.BA", "OEST.BA", "PATA.BA", "RICH.BA", "RIGO.BA", "SAMI.BA", "SEMI.BA"]

    cedears = ["AAL", "AAPL", "ABBV", "ABEV", "ABNB", "ABT", "ADBE", "ADGO", "ADI", "ADP", "AEM", "AIG", "AMAT", "AMD", 
               "AMGN", "AMZN", "AOCA", "ARCO", "ARKK", "ASR", "AUY", "AVGO", "AXP", "AZN", "BA", "BA.C", "BABA", "BB", 
               "BBD", "BBV", "BCS", "BHP", "BIDU", "BIIB", "BIOX", "BITF", "BK", "BMY", "BNG", "BP", "BRFS", "BRKB", "BSBR", 
               "C", "CAAP", "CAH", "CAR", "CAT", "CBRD", "CDE", "CL", "COIN", "COST", "CRM", "CS", "CSCO", "CVX", "CX", "DD", 
               "DE", "DESP", "DIA", "DISN", "DOCU", "DOW", "E", "EA", "EBAY", "EEM", "EFX", "ERIC", "ERJ", "ETSY", "EWZ", "F", 
               "FCX", "FDX", "FMX", "FSLR", "GE", "GFI", "GGB", "GILD", "GLOB", "GLW", "GM", "GOLD", "GOOGL", "GPRK", "GRMN", 
               "GS", "HAL", "HD", "HL", "HMC", "HMY", "HOG", "HON", "HPQ", "HSBC", "HSY", "HUT", "HWM", "IBM", "IFF", "INTC", 
               "ITUB", "IWM", "JD", "JMIA", "JNJ", "JPM", "KMB", "KO", "KOFM", "LLY", "LMT", "LRCX", "LVS", "LYG", "MA", "MCD", 
               "MDT", "MELI", "META", "MMM", "MO", "MOS", "MRK", "MSFT", "MSI", "MSTR", "MU", "NEM", "NFLX", "NGG", "NIO", "NKE", 
               "NOKA", "NTCO", "NTES", "NUE", "NVDA", "NVS", "ORAN", "ORCL", "OXY", "PAAS", "PAC", "PANW", "PBI", "PBR", "PCAR", 
               "PEP", "PFE", "PG", "PHG", "PKS", "PSX", "PYPL", "QCOM", "QQQ", "RBLX", "RIO", "RTX", "SAN", "SAP", "SATL", "SBUX", 
               "SCCO", "SE", "SHEL", "SHOP", "SI", "SID", "SLB", "SNAP", "SNOW", "SONY", "SPGI", "SPOT", "SPY", "SQ", "SYY", "T", 
               "TEFO", "TEN", "TGT", "TM", "TMO", "TRIP", "TRVV", "TSLA", "TSM", "TTE", "TV", "TWLO", "TXN", "TXR", "UAL", "UBER", 
               "UGP", "UL", "UNH", "UNP", "UPST", "USB", "V", "VALE", "VIST", "VIV", "VOD", "VZ", "WBA", "WFC", "WMT", "X", "XLE", 
               "XLF", "XOM", "XP", "YY", "ZM"]

    adrs = ["BBAR", "BMA", "CEPU", "CRESY", "EDN", "GGAL", "IRS", "LOMA", "PAM", "SUPV", "TEO", "TGS", "TS", "TX", "YPF"]

    sectors = ["XLC", "XLP", "XLY", "XLF", "XLV", "XLI", "XLRE", "XLU", "XBI", "XLB", "XLK", "XLE"]
    
    precarga = ["lideres", "general", "cedears", "adrs", "sectores"]
    precarga_dict = {"lideres" : lideres_arg, "general" : general_arg, "cedears" : cedears, "adrs" : adrs, "sectores" : sectors}
    
    if activos in precarga:
        activos = precarga_dict[activos]
    
    if tipo == "no end":
        data = yf.download(activos, start = data_from, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "end":
        data = yf.download(activos, start = data_from, end = data_to, interval = interval, progress = False, auto_adjust = True)
    elif tipo == "period":
        data = yf.download(activos, interval = interval, period= period, progress = False, auto_adjust = True)
    
    if swap:
        #data = data.swaplevel(i = 0, j = 1, axis = 1)
        # Algoritmo para procesar el MultipleTicker download de yfinance
        dicto = {}
        low = data["Low"]
        high = data["High"]
        close = data["Close"]
        open = data["Open"]
        volume = data["Volume"]

        tickers = list(data["Close"].columns)

        for ticker in tickers:
            dicto[ticker] = {
                "Open" : open[ticker],
                "High" : high[ticker],
                "Low" : low[ticker],
                "Close" : close[ticker],
                "Volume" : volume[ticker]
            }

            dicto[ticker] = pd.DataFrame(dicto[ticker])
        return dicto
    return data

In [2]:
# SUBSISTEMA GENERAL (multi timeframe):

def analyzeMACD(activos, timeframe):
    """
    Esta función analiza los activos solicitados con el indicador técnico MACD y calcula la tendencia, dominancia y fortaleza de la 
    dominancia. Definiciones:
        - "Tendencia": si la línea MACD es mayor a 0, la tendencia es alcista, y si la misma es menor a 0, es bajista.
        - "Dominancia": si la línea MACD es mayor a la línea señal, la dominancia es mayor a 0 (histograma positivo), y viceversa.
        - "Fortaleza": - Si la dominancia es alcista: si la diferencia entre las líneas MACD y señal se hace más positiva,
                       la fortaleza es Creciente, y sino, Decreciente.
                       - Si la dominancia es bajista: si la diferencia entre las líneas MACD y señal se hace más negativa,
                       la fortaleza es Creciente, y sino, Decreciente.
                       
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]. También hay sets de activos preconfigurados en la función:
            - "lideres": si se desea cargar todo el panel lider de Argentina;
            - "general": si se desea cargar todo el panel general de Argentina;
            - "adrs": si se desea cargar todo el listado de ADRs argentinos (en usd);
            - "sectors": si se desea cargar el listado de sectores de la economía (ETFs representativos de USA);
            - "cedears": si se desea cargar todo el listado de certificados de depósito de activos del exterior que cotizan en Argentina (puede demorar bastante).
        - timeframe: es la duración de cada sesión/vela (compresión temporal). Puede ser "diario", "semanal" o "mensual".
        
    # Outputs:
        - tabla_final_activos_en_filas: es un DF resumen con los activos en las filas y las variables en las columnas.
        - tabla_final_activos_en_columnas: es un DF resumen con los activos en las columnas y las variables en las filas.
        - df_macd: es un DF con los valores numéricos de la línea MACD (EMA12 - EMA26)
        - df_histoMACD: es un DF con los valores numéricos de la diferencia entre la línea MACD y la línea señal (EMA de 9 períodos de macd)
    """
    
    import pandas as pd
    import pandas_ta as ta
    import yfinance as yf
    
    if timeframe == "diario":
        interval = "1d"
        period = "1y"
    elif timeframe == "semanal":
        interval = "1wk"
        period = "2y"
    elif timeframe == "mensual":
        interval = "1mo"
        period = "4y"
    else:
        return "timeframe incorrecto, elegir 'diario', 'semanal' o 'mensual'"
    
    data = getDataYfMulti(activos, tipo = "period", interval = interval, period = period, swap = False)["Close"]
    data = pd.DataFrame(data)
    data.dropna(inplace = True)

    # Defino datos de estrategia
    MyStrategy = ta.Strategy(
        name = f"MACDM{timeframe}",
        ta = [
            {"kind" : "macd"},
        ]
    )

    df_macd = pd.DataFrame(index = data.index)             # lleva línea macd (interesa el valor respecto a 0) => Indica Tendencia (Alcista / Bajista)
    df_histoMACD = pd.DataFrame(index = data.index)        # lleva histoMACD (interesa la pendiente) => Indica Fortaleza (Creciente / Decreciente)
    df_crucesVigentes = pd.DataFrame(index = data.index)   # lleva histoMACD (interesa el signo) => Indica Dominancia (Alcista / Bajista)

    tendencia = pd.DataFrame()
    fortaleza = pd.DataFrame()
    dominancia = pd.DataFrame()

    df_resumen = pd.DataFrame() # ==> Resumen de todos los activos listados en una sola tabla.
    dict_resumenPorActivo = {}  # ==> Resumen por activo (un diccionario de diccionarios).


    renames = {"MACD_12_26_9" : "macd", "MACDh_12_26_9" : "histoMACD", "MACDs_12_26_9" : "signal"}
    i = 0
    
    for activo in data.columns:
        try:
            datat = pd.DataFrame(data[activo])
            datat.rename(columns = {activo : "Close"}, inplace = True)   # Tengo que renombrar el ticker por "Close", sino no se calcula la MACD.
            datat.ta.strategy(MyStrategy)
            datat.rename(columns = renames, inplace = True)
            datat.dropna(inplace = True)

            df_macd[activo] = datat["macd"].dropna()  # <== OK!!!         # DF que contiene todos los valores de la línea "macd" de cada activo.
            df_histoMACD[activo] = datat["histoMACD"] # <== OK!!!
            df_crucesVigentes[activo] = datat["histoMACD"]   # <== OK!!!

            df_macd = df_macd.dropna()
            df_histoMACD = df_histoMACD.dropna() 
            df_crucesVigentes = df_crucesVigentes.dropna()


            if df_macd[activo][-1] >= 0:
                dict_resumenPorActivo[activo] = {"tendencia" : "Alcista"}
            elif df_macd[activo][-1] < 0:
                dict_resumenPorActivo[activo] = {"tendencia" : "Bajista"}

            if df_crucesVigentes[activo][-1] >= 0:
                dict_resumenPorActivo[activo]["dominancia"] = "Alcista"
            elif df_crucesVigentes[activo][-1] < 0:
                dict_resumenPorActivo[activo]["dominancia"] = "Bajista"


            if df_crucesVigentes[activo][-1] >= 0:  # Si la dominancia es alcista...
                if df_crucesVigentes[activo][-1] >= df_crucesVigentes[activo][-2]:
                    dict_resumenPorActivo[activo]["fortaleza"] = "Creciente"
                elif df_crucesVigentes[activo][-1] < df_crucesVigentes[activo][-2]:
                    dict_resumenPorActivo[activo]["fortaleza"] = "Decreciente"

            if df_crucesVigentes[activo][-1] < 0:  # Si la dominancia es bajista...
                if df_crucesVigentes[activo][-1] <= df_crucesVigentes[activo][-2]:
                    dict_resumenPorActivo[activo]["fortaleza"] = "Creciente"
                elif df_crucesVigentes[activo][-1] > df_crucesVigentes[activo][-2]:
                    dict_resumenPorActivo[activo]["fortaleza"] = "Decreciente"


            tendencia[f"tend_{activo}"] = np.where(df_macd[activo] > 0, "T. Alcista", "T. Bajista")  # <== OJO, los NaN los toma como  Bajista
            dominancia[f"domin_{activo}"] = np.where(df_crucesVigentes[activo] > 0, "D. Alcista", "D. Bajista")  #  <== OJO, los NaN los toma como Bajista
            fortaleza[f"fort_{activo}"] = np.where(df_crucesVigentes[activo] > df_crucesVigentes[activo].shift(1), "Creciente", "Decreciente")

            tendencia.set_index(df_macd.index, inplace = True)
            dominancia.set_index(df_crucesVigentes.index, inplace = True)
            fortaleza.set_index(df_crucesVigentes.index, inplace = True)

            df_resumen[f"tend_{activo}"] = tendencia[f"tend_{activo}"]
            df_resumen[f"domin_{activo}"] = dominancia[f"domin_{activo}"]
            df_resumen[f"fort_{activo}"] = fortaleza[f"fort_{activo}"]

            tabla_final_activos_en_columnas = pd.DataFrame(dict_resumenPorActivo)
            tabla_final_activos_en_filas = tabla_final_activos_en_columnas.transpose()
            i += 1
            print(f"[TF {timeframe}] Procesando {i} de {len(data.columns)} activos ...")

        except:
            pass
    
    def set_styles(tabla):
        def highlight_tendDomin(val, color_if_true, color_if_false):
            color = color_if_true if val == "Alcista" else color_if_false
            return f"background-color: {color}"

        highlighted_rows = np.where((tabla['dominancia'] == "Alcista") & (tabla['fortaleza'] == "Creciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['dominancia'] == "Alcista") & (tabla['fortaleza'] == "Decreciente"), 
                                    'background-color: #FA6B84', 
                                    np.where((tabla['dominancia'] == "Bajista") & (tabla['fortaleza'] == "Decreciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['dominancia'] == "Bajista") & (tabla['fortaleza'] == "Creciente"), 
                                    'background-color: #FA6B84', ''))))

        styler = tabla.style.apply(lambda _: highlighted_rows, subset=["fortaleza"]) \
                            .applymap(highlight_tendDomin, color_if_true='#90EF90', color_if_false='#FA6B84', 
                              subset=['tendencia', 'dominancia']) \
                            .applymap(lambda _: "color: black; font-weight: bold")

        return styler
    
    tabla_est = set_styles(tabla_final_activos_en_filas)
    
    
    #dict_to_df_activos_en_filas.to_excel(f"MACD{activos}-tf{timeframe}.xlsx")  
    return tabla_final_activos_en_filas, tabla_final_activos_en_columnas, df_macd, df_histoMACD, tabla_est

def analyzeFullMACD(activos):
    """
    Esta función se apoya en la función analyze MACD para generar un análisis multitemporal de los activos especificados.
    Los timeframes analizados son: mensual, semanal y diario.
    
    Analiza los activos solicitados con el indicador técnico MACD y calcula la tendencia, dominancia y fortaleza de la 
    dominancia. Definiciones:
        - "Tendencia": si la línea MACD es mayor a 0, la tendencia es alcista, y si la misma es menor a 0, es bajista.
        - "Dominancia": si la línea MACD es mayor a la línea señal, la dominancia es mayor a 0 (histograma positivo), y viceversa.
        - "Fortaleza": - Si la dominancia es alcista: si la diferencia entre las líneas MACD y señal se hace más positiva,
                       la fortaleza es Creciente, y sino, Decreciente.
                       - Si la dominancia es bajista: si la diferencia entre las líneas MACD y señal se hace más negativa,
                       la fortaleza es Creciente, y sino, Decreciente.
                       
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]
        - timeframe: es la duración de cada sesión/vela (compresión temporal). Puede ser "diario", "semanal" o "mensual".
        
    # Outputs:
        - data: una tabla con los activos en las filas y la tendencia, dominancia y fortaleza de la dominancia para cada
        plano temporal (mensual, semanal y diario) como columnas.
    """
    
    diario = analyzeMACD(activos = activos, timeframe = "diario")[0]
    semanal = analyzeMACD(activos = activos, timeframe = "semanal")[0]
    mensual = analyzeMACD(activos = activos, timeframe = "mensual")[0]
    
    diario.rename(columns = {"tendencia" : "tendencia_1D", "dominancia" : "dominancia_1D", 
                             "fortaleza" : "fortaleza_1D"}, inplace = True)
    
    semanal.rename(columns = {"tendencia" : "tendencia_1W", "dominancia" : "dominancia_1W", 
                             "fortaleza" : "fortaleza_1W"}, inplace = True)
    
    mensual.rename(columns = {"tendencia" : "tendencia_1M", "dominancia" : "dominancia_1M", 
                             "fortaleza" : "fortaleza_1M"}, inplace = True)
    data = pd.concat([mensual, semanal, diario], axis = 1)
    
    def set_styles(tabla):
        def highlight_tendDomin(val, color_if_true, color_if_false):
            color = color_if_true if val == "Alcista" else color_if_false
            return f"background-color: {color}"

        highlighted_rows_M = np.where((tabla['dominancia_1M'] == "Alcista") & (tabla['fortaleza_1M'] == "Creciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['dominancia_1M'] == "Alcista") & (tabla['fortaleza_1M'] == "Decreciente"), 
                                    'background-color: #FA6B84', 
                                    np.where((tabla['dominancia_1M'] == "Bajista") & (tabla['fortaleza_1M'] == "Decreciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['dominancia_1M'] == "Bajista") & (tabla['fortaleza_1M'] == "Creciente"), 
                                    'background-color: #FA6B84', ''))))

        highlighted_rows_W = np.where((tabla['dominancia_1W'] == "Alcista") & (tabla['fortaleza_1W'] == "Creciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['dominancia_1W'] == "Alcista") & (tabla['fortaleza_1W'] == "Decreciente"), 
                                    'background-color: #FA6B84', 
                                    np.where((tabla['dominancia_1W'] == "Bajista") & (tabla['fortaleza_1W'] == "Decreciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['dominancia_1W'] == "Bajista") & (tabla['fortaleza_1W'] == "Creciente"), 
                                    'background-color: #FA6B84', ''))))

        highlighted_rows_D = np.where((tabla['dominancia_1D'] == "Alcista") & (tabla['fortaleza_1D'] == "Creciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['dominancia_1D'] == "Alcista") & (tabla['fortaleza_1D'] == "Decreciente"), 
                                    'background-color: #FA6B84', 
                                    np.where((tabla['dominancia_1D'] == "Bajista") & (tabla['fortaleza_1D'] == "Decreciente"),
                                    'background-color: #90EF90',
                                    np.where((tabla['dominancia_1D'] == "Bajista") & (tabla['fortaleza_1D'] == "Creciente"), 
                                    'background-color: #FA6B84', ''))))


        styler = tabla.style.apply(lambda _: highlighted_rows_M, subset=["fortaleza_1M"]) \
                            .apply(lambda _: highlighted_rows_W, subset=["fortaleza_1W"]) \
                            .apply(lambda _: highlighted_rows_D, subset=["fortaleza_1D"]) \
                            .applymap(highlight_tendDomin, color_if_true='#90EF90', color_if_false='#FA6B84', 
                              subset=['tendencia_1M', 'tendencia_1W', 'tendencia_1D', 'dominancia_1M', 'dominancia_1W', 'dominancia_1D']) \
                            .applymap(lambda _: "color: black; font-weight: bold")

        return styler
    return set_styles(data)
    
    
def calculateMACD(data):
    """
    Función sencilla que permite agregar el cálculo del indicador técnico MACD a un dataframe tipo OHLC. Se requiere una
    columna de precios de cierre llamada "Close" ordenado en forma ascendente por fecha. Devuelve un dataframe con columnas 
    nuevas: "EMA12", "EMA26", "macd", "signal" e "histoMACD".
    
    # Inputs:
        - data: dataframe con columna "Close", ordenado en forma ascendente por fecha.
    # Outputs:
        - data: dataframe con el indicador técnico MACD calculado. Devuelve un dataframe con columnas nuevas: 
        "EMA12", "EMA26", "macd", "signal" e "histoMACD".
    """
    
    data["EMA12"] = data["Close"].ewm(span = 12).mean()
    data["EMA26"] = data["Close"].ewm(span = 26).mean()
    data["macd"] = data["EMA12"] - data["EMA26"]
    data["signal"] = data["macd"].ewm(span = 9).mean()
    data["histoMACD"] = data["macd"] - data["signal"]
    
    return data

def getMACDSignals(activos, timeframe):
    """
    Esta función descarga data financiera de los activos seleccionados, calcula el indicador técnico MACD e identifica las 
    últimas señales vigentes de compra (long) o bien de venta en corto (short), devolviendo la fecha en que se produjo la 
    señal, el precio, el rendimiento acumulado al día de la consulta y el rendimiento anualizado. Estos datos se devuelven 
    en formato diccionario o dataframe (tabla) para todos los activos consultados, este último ordenado por fecha reciente 
    de señales. Las señales son en función de dos tipos de criterios:
        - Cruce de la línea MACD de la posición 0, que es equivalente a un crossover de las medias móviles exponenciales de 
        12 y 26 ruedas;
        - Cruce de la línea MACD a la línea Señal (crossover), siendo la línea Señal una EMA de 9 ruedas de la línea MACD.
                       
    # Inputs:
        - activos: es una lista de activos a analizar. Ej.: adrs = ["GGAL", "BBAR", "LOMA", "CEPU"]. También hay sets de activos 
        preconfigurados en la función:
            - "lideres": si se desea cargar todo el panel lider de Argentina;
            - "general": si se desea cargar todo el panel general de Argentina;
            - "adrs": si se desea cargar todo el listado de ADRs argentinos (en usd);
            - "sectors": si se desea cargar el listado de sectores de la economía (ETFs representativos de USA);
            - "cedears": si se desea cargar todo el listado de certificados de depósito de activos del exterior que cotizan en 
            Argentina (puede demorar bastante).
        - timeframe: es la duración de cada sesión/vela (compresión temporal). Puede ser "diario", "semanal" o "mensual".
        
    # Outputs: macd_pos0_signals, macd_cross_signals, macd_pos0_signals_df, macd_cross_signals_df
        - macd_pos0_signals: es un diccionario con las señales según el criterio "Cruce de la línea MACD de la posición 0";
        - macd_cross_signals: es un diccionario con las señales según el criterio "Cruce de la línea MACD a la línea Señal (crossover)";
        - macd_pos0_signals_df: es el macd_pos0_signals en formato tabla/dataframe.
        - macd_cross_signals_df: es el macd_cross_signals en formato tabla/dataframe.
        - macd_pos0_signals_df_formateado: es lo anterior pero formateado con una escala de colores según rendimiento acumulado.
        - macd_cross_signals_df_formateado: es lo anterior pero formateado con una escala de colores según rendimiento acumulado.
        
        Todos los outputs contienen los siguientes atributos: type (long/short), 
                                                              precioSeñal (precio de la señal), 
                                                              fechaSeñal (fecha de la señal),
                                                              diasDesdeSeñal (dias transcurridos desde la señal),
                                                              rendAcumDesdeSeñal (rendimiento acumulado desde la señal),
                                                              rendAnualDesdeSeñal (rendimiento acumulado anualizado);
    """
    
    import pandas as pd
    import pandas_ta as ta
    import numpy as np
    import yfinance as yf
    from datetime import datetime
    
    if timeframe == "diario":
        interval = "1d"
        period = "1y"
    elif timeframe == "semanal":
        interval = "1wk"
        period = "2y"
    elif timeframe == "mensual":
        interval = "1mo"
        period = "4y"
    else:
        return "timeframe incorrecto, elegir 'diario', 'semanal' o 'mensual'"
    
    # Obtengo la data financiera
    data = getDataYfMulti(activos, tipo = "period", interval = interval, period = period, swap = False)["Close"]
    data = pd.DataFrame(data)
    data.dropna(inplace = True)

    # Defino datos de estrategia
    MyStrategy = ta.Strategy(
            name = f"MACD{timeframe}",
            ta = [
                {"kind" : "macd"},
            ]
    )


    dict_resumenPorActivo = {}  # ==> Resumen por activo (un diccionario de diccionarios).
    renames = {"MACD_12_26_9" : "macd", "MACDh_12_26_9" : "histoMACD", "MACDs_12_26_9" : "signal"}
    macd_pos0_signals = {}
    macd_cross_signals = {}

    for activo in data.columns:
        try:
            # Iteramos por cada activo
            # Armamos el dataframe OHLC y calculamos componentes del MACD
            datat = pd.DataFrame(data[activo])
            datat.rename(columns = {activo : "Close"}, inplace = True)   # Tengo que renombrar el ticker por "Close", sino no se calcula la MACD.
            datat.ta.strategy(MyStrategy)
            datat.rename(columns = renames, inplace = True)
            datat.dropna(inplace = True)
            
            # Obtenemos el precio actual
            actual_price = datat["Close"][-1]

            # I) TIPO: Cambios de la línea MACD respecto a 0 (diferencia EMAS 12-26)
            datat["macd_pos0"] = np.where(datat["macd"] > 0, 1, -1)
            datat["macd_change"] = np.where(datat["macd_pos0"] > datat["macd_pos0"].shift(1), "Compra", 
                                            np.where(datat["macd_pos0"] < datat["macd_pos0"].shift(1), "Venta", ""))

            
            # I.1) Caso de Long, calculamos estadísticas: 
            if datat["macd_pos0"][-1] > 0:
                last_buy_signal_0 = datat[datat["macd_change"] == "Compra"].iloc[-1]
                price_lbs_0 = round(last_buy_signal_0["Close"],2)
                date_lbs_0 = datetime(last_buy_signal_0.name.year, 
                                      last_buy_signal_0.name.month, 
                                      last_buy_signal_0.name.day)
                days_since_bs_0 = (datetime.today() - date_lbs_0).days
                rend_since_bs_0 = round(((actual_price / price_lbs_0) - 1) * 100, 2)
                rend_since_bs_annualized_0 = round((((1 + ((actual_price / price_lbs_0) - 1)) ** (365/days_since_bs_0)) - 1) * 100,2)
                macd_pos0_signals[activo] = {"type" : "long", 
                                             "precioSeñal" : price_lbs_0, 
                                             "fechaSeñal": date_lbs_0.strftime("%Y-%m-%d"), 
                                             "diasDesdeSeñal" : days_since_bs_0,
                                             "rendAcumDesdeSeñal":  rend_since_bs_0, 
                                             "rendAnualDesdeSeñal" : rend_since_bs_annualized_0}
                

            # I.2) Caso de Short, calculamos estadísticas: 
            elif datat["macd_pos0"][-1] < 0:
                last_sell_signal_0 = datat[datat["macd_change"] == "Venta"].iloc[-1]
                price_lss_0 = round(last_sell_signal_0["Close"],2)
                date_lss_0 = datetime(last_sell_signal_0.name.year, 
                                      last_sell_signal_0.name.month, 
                                      last_sell_signal_0.name.day)
                days_since_ss_0 = (datetime.today() - date_lss_0).days
                rend_since_ss_0 = round(((price_lss_0 / actual_price) - 1) * 100, 2)
                rend_since_ss_annualized_0 = round((((1 + ((price_lss_0 / actual_price) - 1)) ** (365/days_since_ss_0)) - 1) * 100,2)
                macd_pos0_signals[activo] = {"type" : "short", 
                                             "precioSeñal" : price_lss_0, 
                                             "fechaSeñal": date_lss_0.strftime("%Y-%m-%d"), 
                                             "diasDesdeSeñal" : days_since_ss_0,
                                             "rendAcumDesdeSeñal":  rend_since_ss_0, 
                                             "rendAnualDesdeSeñal" : rend_since_ss_annualized_0}


            # II) TIPO: Cruces de línea MACD con Línea Señal:
            datat["histoMACD_pos0"] = np.where(datat["histoMACD"] > 0, 1, -1)
            datat["histoMACD_change"] = np.where(datat["histoMACD_pos0"] > datat["histoMACD_pos0"].shift(1), "Compra", 
                                            np.where(datat["histoMACD_pos0"] < datat["histoMACD_pos0"].shift(1), "Venta", ""))

            # II.1) Caso de Long, calculamos estadísticas:
            if datat["histoMACD_pos0"][-1] > 0:
                last_buy_signal_crossover = datat[datat["histoMACD_change"] == "Compra"].iloc[-1]
                price_lbs_crossover = round(last_buy_signal_crossover["Close"],2)
                date_lbs_crossover = datetime(last_buy_signal_crossover.name.year, 
                                              last_buy_signal_crossover.name.month, 
                                              last_buy_signal_crossover.name.day)
                days_since_bs_crossover = (datetime.today() - date_lbs_crossover).days
                rend_since_bs_crossover = round(((actual_price / price_lbs_crossover) - 1) * 100, 2)
                rend_since_bs_annualized_crossover = round((((1 + ((actual_price / price_lbs_crossover) - 1)) ** (365/days_since_bs_crossover)) - 1) * 100,2)
                macd_cross_signals[activo] = {"type" : "long", 
                                              "precioSeñal" : price_lbs_crossover, 
                                              "fechaSeñal": date_lbs_crossover.strftime("%Y-%m-%d"), 
                                              "diasDesdeSeñal" : days_since_bs_crossover,
                                             "rendAcumDesdeSeñal":  rend_since_bs_crossover, 
                                              "rendAnualDesdeSeñal" : rend_since_bs_annualized_crossover}

            # II.2) Caso de Short, calculamos estadísticas:     
            elif datat["histoMACD_pos0"][-1] < 0:
                last_sell_signal_crossover = datat[datat["histoMACD_change"] == "Venta"].iloc[-1]
                price_lss_crossover = round(last_sell_signal_crossover["Close"],2)
                date_lss_crossover = datetime(last_sell_signal_crossover.name.year, 
                                              last_sell_signal_crossover.name.month, 
                                              last_sell_signal_crossover.name.day)
                days_since_ss_crossover = (datetime.today() - date_lss_crossover).days
                rend_since_ss_crossover = round(((price_lss_crossover / actual_price) - 1) * 100, 2)
                rend_since_ss_annualized_crossover = round((((1 + ((price_lss_crossover / actual_price) - 1)) ** (365/days_since_ss_crossover)) - 1) * 100,2)
                macd_cross_signals[activo] = {"type" : "short", 
                                              "precioSeñal" : price_lss_crossover, 
                                              "fechaSeñal": date_lss_crossover.strftime("%Y-%m-%d"), 
                                              "diasDesdeSeñal" : days_since_ss_crossover,
                                             "rendAcumDesdeSeñal":  rend_since_ss_crossover, 
                                              "rendAnualDesdeSeñal" : rend_since_ss_annualized_crossover}
            print(f"{activo} - procesado!")
        except:
            print(f"{activo} - no pudo ser procesado")
            pass
    
    macd_pos0_signals_df = pd.DataFrame(macd_pos0_signals).transpose().sort_values(by = "diasDesdeSeñal", ascending = True)
    macd_cross_signals_df = pd.DataFrame(macd_cross_signals).transpose().sort_values(by = "diasDesdeSeñal", ascending = True)
    
    macd_pos0_signals_df_formateado = macd_pos0_signals_df.style.background_gradient(axis=0, gmap=macd_pos0_signals_df['rendAnualDesdeSeñal'], 
                                                                                     cmap='RdYlGn') \
    .format('{:.2f}%', subset=["rendAcumDesdeSeñal", "rendAnualDesdeSeñal"]) \
    .format('{:.2f}', subset=["precioSeñal"])
    
    macd_cross_signals_df_formateado = macd_cross_signals_df.style.background_gradient(axis=0, gmap=macd_cross_signals_df['rendAnualDesdeSeñal'], 
                                                                                       cmap='RdYlGn') \
    .format('{:.2f}%', subset=["rendAcumDesdeSeñal", "rendAnualDesdeSeñal"]) \
    .format('{:.2f}', subset=["precioSeñal"])
    
    return [macd_pos0_signals, macd_cross_signals, 
            macd_pos0_signals_df, macd_cross_signals_df,
            macd_pos0_signals_df_formateado, macd_cross_signals_df_formateado]

In [3]:
datos = analyzeFullMACD(activos = "adrs")

[TF diario] Procesando 1 de 15 activos ...
[TF diario] Procesando 2 de 15 activos ...
[TF diario] Procesando 3 de 15 activos ...
[TF diario] Procesando 4 de 15 activos ...
[TF diario] Procesando 5 de 15 activos ...
[TF diario] Procesando 6 de 15 activos ...
[TF diario] Procesando 7 de 15 activos ...
[TF diario] Procesando 8 de 15 activos ...
[TF diario] Procesando 9 de 15 activos ...
[TF diario] Procesando 10 de 15 activos ...
[TF diario] Procesando 11 de 15 activos ...
[TF diario] Procesando 12 de 15 activos ...
[TF diario] Procesando 13 de 15 activos ...
[TF diario] Procesando 14 de 15 activos ...
[TF diario] Procesando 15 de 15 activos ...
[TF semanal] Procesando 1 de 15 activos ...
[TF semanal] Procesando 2 de 15 activos ...
[TF semanal] Procesando 3 de 15 activos ...
[TF semanal] Procesando 4 de 15 activos ...
[TF semanal] Procesando 5 de 15 activos ...
[TF semanal] Procesando 6 de 15 activos ...
[TF semanal] Procesando 7 de 15 activos ...
[TF semanal] Procesando 8 de 15 activos .

In [4]:
# analyzeFullMACD - output - tabla resumen
datos

,tendencia_1M,dominancia_1M,fortaleza_1M,tendencia_1W,dominancia_1W,fortaleza_1W,tendencia_1D,dominancia_1D,fortaleza_1D
BBAR,Alcista,Alcista,Creciente,Alcista,Alcista,Creciente,Bajista,Alcista,Creciente
BMA,Alcista,Alcista,Creciente,Alcista,Alcista,Creciente,Alcista,Alcista,Creciente
CEPU,Alcista,Alcista,Creciente,Alcista,Alcista,Decreciente,Alcista,Bajista,Decreciente
CRESY,Alcista,Alcista,Creciente,Alcista,Alcista,Decreciente,Bajista,Bajista,Decreciente
EDN,Alcista,Alcista,Creciente,Alcista,Alcista,Decreciente,Alcista,Bajista,Decreciente
GGAL,Alcista,Alcista,Creciente,Alcista,Alcista,Creciente,Alcista,Alcista,Creciente
IRS,Alcista,Alcista,Creciente,Alcista,Alcista,Creciente,Bajista,Alcista,Creciente
LOMA,Alcista,Alcista,Creciente,Alcista,Alcista,Creciente,Alcista,Alcista,Creciente
PAM,Alcista,Alcista,Decreciente,Alcista,Alcista,Decreciente,Alcista,Bajista,Decreciente
SUPV,Alcista,Alcista,Creciente,Alcista,Alcista,Decreciente,Alcista,Alcista,Creciente


In [5]:
tabla, _, df_macd, df_hMACD, tabla_est = analyzeMACD(activos = "adrs", timeframe = "semanal")

[TF semanal] Procesando 1 de 15 activos ...
[TF semanal] Procesando 2 de 15 activos ...
[TF semanal] Procesando 3 de 15 activos ...
[TF semanal] Procesando 4 de 15 activos ...
[TF semanal] Procesando 5 de 15 activos ...
[TF semanal] Procesando 6 de 15 activos ...
[TF semanal] Procesando 7 de 15 activos ...
[TF semanal] Procesando 8 de 15 activos ...
[TF semanal] Procesando 9 de 15 activos ...
[TF semanal] Procesando 10 de 15 activos ...
[TF semanal] Procesando 11 de 15 activos ...
[TF semanal] Procesando 12 de 15 activos ...
[TF semanal] Procesando 13 de 15 activos ...
[TF semanal] Procesando 14 de 15 activos ...
[TF semanal] Procesando 15 de 15 activos ...


In [6]:
# analyzeMACD - output - tabla resumen TF semanal
tabla_est

,tendencia,dominancia,fortaleza
BBAR,Alcista,Alcista,Creciente
BMA,Alcista,Alcista,Creciente
CEPU,Alcista,Alcista,Decreciente
CRESY,Alcista,Alcista,Decreciente
EDN,Alcista,Alcista,Decreciente
GGAL,Alcista,Alcista,Creciente
IRS,Alcista,Alcista,Creciente
LOMA,Alcista,Alcista,Creciente
PAM,Alcista,Alcista,Decreciente
SUPV,Alcista,Alcista,Decreciente


In [11]:
(macd_pos0_signals, macd_cross_signals,
macd_pos0_signals_df, macd_cross_signals_df, macd_pos0_signals_df_formateado,
macd_cross_signals_df_formateado) = getMACDSignals(activos = "adrs", timeframe = "semanal")

BBAR - procesado!
BMA - procesado!
CEPU - procesado!
CRESY - procesado!
EDN - procesado!
GGAL - procesado!
IRS - procesado!
LOMA - procesado!
PAM - no pudo ser procesado
SUPV - procesado!
TEO - procesado!
TGS - procesado!
TS - procesado!
TX - procesado!
YPF - procesado!


In [14]:
# getMACDSignals - output - tabla con estilos de las señales de cruce entre la línea MACD (rápida) y el nivel 0 (equivalente a crossover EMA12-26)
macd_pos0_signals_df_formateado

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,rendAcumDesdeSeñal,rendAnualDesdeSeñal
SUPV,long,3.84,2023-12-04,50,8.85%,85.77%
TX,long,39.12,2023-12-04,50,-1.20%,-8.45%
BBAR,long,5.48,2023-11-27,57,1.82%,12.28%
GGAL,long,16.93,2023-11-27,57,7.38%,57.80%
IRS,long,8.93,2023-11-27,57,-2.69%,-16.01%
TGS,long,13.37,2023-11-27,57,10.55%,90.03%
CEPU,long,7.46,2023-11-20,64,24.53%,249.44%
EDN,long,16.40,2023-11-20,64,23.72%,236.66%
LOMA,long,6.50,2023-11-20,64,12.62%,96.91%
YPF,long,16.84,2023-11-20,64,-7.13%,-34.40%


In [15]:
# getMACDSignals - output - tabla con estilos de las señales de cruce entre las líneas macd y señal
macd_cross_signals_df_formateado

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,rendAcumDesdeSeñal,rendAnualDesdeSeñal
TS,short,32.31,2024-01-08,15,1.03%,28.38%
BBAR,long,5.48,2023-11-27,57,1.82%,12.28%
BMA,long,27.09,2023-11-27,57,11.55%,101.41%
EDN,long,16.49,2023-11-27,57,23.04%,277.32%
GGAL,long,16.93,2023-11-27,57,7.38%,57.80%
IRS,long,8.93,2023-11-27,57,-2.69%,-16.01%
LOMA,long,6.62,2023-11-27,57,10.57%,90.34%
SUPV,long,3.39,2023-11-27,57,23.30%,282.45%
TGS,long,13.37,2023-11-27,57,10.55%,90.03%
TX,long,40.34,2023-11-27,57,-4.19%,-23.97%


In [16]:
# getMACDSignals - output - macd_pos0_signals
macd_pos0_signals

{'BBAR': {'type': 'long',
  'precioSeñal': 5.48,
  'fechaSeñal': '2023-11-27',
  'diasDesdeSeñal': 57,
  'rendAcumDesdeSeñal': 1.82,
  'rendAnualDesdeSeñal': 12.28},
 'BMA': {'type': 'long',
  'precioSeñal': 14.06,
  'fechaSeñal': '2022-12-19',
  'diasDesdeSeñal': 400,
  'rendAcumDesdeSeñal': 114.94,
  'rendAnualDesdeSeñal': 101.02},
 'CEPU': {'type': 'long',
  'precioSeñal': 7.46,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'rendAcumDesdeSeñal': 24.53,
  'rendAnualDesdeSeñal': 249.44},
 'CRESY': {'type': 'long',
  'precioSeñal': 6.23,
  'fechaSeñal': '2023-01-02',
  'diasDesdeSeñal': 386,
  'rendAcumDesdeSeñal': 48.96,
  'rendAnualDesdeSeñal': 45.76},
 'EDN': {'type': 'long',
  'precioSeñal': 16.4,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'rendAcumDesdeSeñal': 23.72,
  'rendAnualDesdeSeñal': 236.66},
 'GGAL': {'type': 'long',
  'precioSeñal': 16.93,
  'fechaSeñal': '2023-11-27',
  'diasDesdeSeñal': 57,
  'rendAcumDesdeSeñal': 7.38,
  'rendAnualDesdeSeñal': 5

In [17]:
# getMACDSignals - output - macd_cross_signals
macd_cross_signals

{'BBAR': {'type': 'long',
  'precioSeñal': 5.48,
  'fechaSeñal': '2023-11-27',
  'diasDesdeSeñal': 57,
  'rendAcumDesdeSeñal': 1.82,
  'rendAnualDesdeSeñal': 12.28},
 'BMA': {'type': 'long',
  'precioSeñal': 27.09,
  'fechaSeñal': '2023-11-27',
  'diasDesdeSeñal': 57,
  'rendAcumDesdeSeñal': 11.55,
  'rendAnualDesdeSeñal': 101.41},
 'CEPU': {'type': 'long',
  'precioSeñal': 7.46,
  'fechaSeñal': '2023-11-20',
  'diasDesdeSeñal': 64,
  'rendAcumDesdeSeñal': 24.53,
  'rendAnualDesdeSeñal': 249.44},
 'CRESY': {'type': 'long',
  'precioSeñal': 7.75,
  'fechaSeñal': '2023-11-13',
  'diasDesdeSeñal': 71,
  'rendAcumDesdeSeñal': 19.74,
  'rendAnualDesdeSeñal': 152.49},
 'EDN': {'type': 'long',
  'precioSeñal': 16.49,
  'fechaSeñal': '2023-11-27',
  'diasDesdeSeñal': 57,
  'rendAcumDesdeSeñal': 23.04,
  'rendAnualDesdeSeñal': 277.32},
 'GGAL': {'type': 'long',
  'precioSeñal': 16.93,
  'fechaSeñal': '2023-11-27',
  'diasDesdeSeñal': 57,
  'rendAcumDesdeSeñal': 7.38,
  'rendAnualDesdeSeñal': 57

In [18]:
# getMACDSignals - output - macd_pos0_signals_df
macd_pos0_signals_df

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,rendAcumDesdeSeñal,rendAnualDesdeSeñal
SUPV,long,3.84,2023-12-04,50,8.85,85.77
TX,long,39.12,2023-12-04,50,-1.2,-8.45
BBAR,long,5.48,2023-11-27,57,1.82,12.28
GGAL,long,16.93,2023-11-27,57,7.38,57.8
IRS,long,8.93,2023-11-27,57,-2.69,-16.01
TGS,long,13.37,2023-11-27,57,10.55,90.03
CEPU,long,7.46,2023-11-20,64,24.53,249.44
EDN,long,16.4,2023-11-20,64,23.72,236.66
LOMA,long,6.5,2023-11-20,64,12.62,96.91
YPF,long,16.84,2023-11-20,64,-7.13,-34.4


In [19]:
# getMACDSignals - output - macd_cross_signals_df
macd_cross_signals_df

,type,precioSeñal,fechaSeñal,diasDesdeSeñal,rendAcumDesdeSeñal,rendAnualDesdeSeñal
TS,short,32.31,2024-01-08,15,1.03,28.38
BBAR,long,5.48,2023-11-27,57,1.82,12.28
BMA,long,27.09,2023-11-27,57,11.55,101.41
EDN,long,16.49,2023-11-27,57,23.04,277.32
GGAL,long,16.93,2023-11-27,57,7.38,57.8
IRS,long,8.93,2023-11-27,57,-2.69,-16.01
LOMA,long,6.62,2023-11-27,57,10.57,90.34
SUPV,long,3.39,2023-11-27,57,23.3,282.45
TGS,long,13.37,2023-11-27,57,10.55,90.03
TX,long,40.34,2023-11-27,57,-4.19,-23.97
